<a href="https://colab.research.google.com/github/AdamsD02/Music-Genre-Classification/blob/main/mgc_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone "https://github.com/AdamsD02/Music-Genre-Classification.git"


Cloning into 'Music-Genre-Classification'...
remote: Enumerating objects: 1015, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 1015 (delta 0), reused 0 (delta 0), pack-reused 1012 (from 2)
Receiving objects: 100% (1015/1015), 1.13 GiB | 25.09 MiB/s, done.
Updating files: 100% (1003/1003), done.


In [2]:
%cd "Music-Genre-Classification"

/content/Music-Genre-Classification


In [3]:
%ls

features_30_sec.csv  genres/  mgc_code.ipynb  README.md


### Install Librosa library for audio analysis and manipulation

In [4]:
pip install librosa

Chatgpt Code

In [6]:
import os
import librosa
import numpy as np
import pandas as pd

# Set path to the GTZAN dataset
DATASET_PATH = "/content/Music-Genre-Classification/genres"
GENRES = os.listdir(DATASET_PATH)

# Store all features
feature_list = []

# Loop over genres and files
for genre in GENRES:
    genre_path = os.path.join(DATASET_PATH, genre)
    for filename in os.listdir(genre_path):
        if filename.endswith('.wav'):
            file_path = os.path.join(genre_path, filename)
            print(file_path)
            y, sr = librosa.load(file_path, duration=30)

            # Extract features
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            zcr = librosa.feature.zero_crossing_rate(y)
            spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
            chroma = librosa.feature.chroma_stft(y=y, sr=sr)

            # Compute mean and variance
            features = {
                "filename": filename,
                "genre": genre,
            }

            # MFCCs
            for i in range(mfcc.shape[0]):
                features[f"mfcc{i+1}_mean"] = np.mean(mfcc[i])
                features[f"mfcc{i+1}_var"] = np.var(mfcc[i])

            # ZCR
            features["zcr_mean"] = np.mean(zcr)
            features["zcr_var"] = np.var(zcr)

            # Spectral Centroid
            features["spec_centroid_mean"] = np.mean(spec_centroid)
            features["spec_centroid_var"] = np.var(spec_centroid)

            # Chroma
            for i in range(chroma.shape[0]):
                features[f"chroma{i+1}_mean"] = np.mean(chroma[i])
                features[f"chroma{i+1}_var"] = np.var(chroma[i])

            feature_list.append(features)
            print(features_lis)

# Save to CSV
# df = pd.DataFrame(feature_list)
# df.to_csv("gtzan_features.csv", index=False)

print("Feature extraction complete. Saved to gtzan_features.csv.")


Feature extraction complete. Saved to gtzan_features.csv.
